# ISA I

## Y86

Y86 is much simpler than x86, there's less instructions, fewer addressing modes, and a simpler system state.

There are various ways of giving *semantics* or meaning to a programming system. The most sensible semantic for assembly launguage is operation semantics or interpreter semantics, which means that we explain the semantics of each possible operation in the language by explaining the effect the operation has on the machine state.

The most fundamental abstraction for machine semantics is **Fetch, Decode, Execute** cycle.

The steps are:  
Fetch: Fetch the next instruction (the PC tells you which instruction is next)  
Decode: Decode the instruction (in the control unit)  
Execute: Execute the instruction, updating the state properly   
Go back to step 1

## Y86 Processor State

The Y86 Processor's **registers** are similar to that in x86: %rax, %rbx, %rcx, %rdx, %rbp, %rsp, %rdi, %rsi, %r8- %r14  
The **condition flags** are one bit flags set by arithmetic and logical flags OF (Overflow), ZF (Zero flag), SF (Negative)   
**Program Counter** indicates the address of instruction   
**Memory** is byte addressable, and words are stored in little endian order   
**Status code** can be AOK, HLT, INS, ADR to indicate state of program execution

We're actually describing both the assembly and machine languages.  
Machine language instructions are read 1-10 bits at a time with the instruction length determined from the first byte  
There aren't as many instruction tyoes and simpler encoding than x86-64  
Each instruction accesses and modifies some part of the program state

Each register has a 4-bit id 0-F, most registers are general purpose except %rsp, which is the stack pointer

## Addressing modes

Immediate: Value to register  
Register: Register to register   
Normal(R)/Indirect Addressing: Data in Memory address in register to register   
Displacement(D) Addressing: Data in (Memory address + amount) in register to register

## Argument Passing

The first six parameters of any procedure are passed in order to 6 specific registers, rdi, rsi, rdx, rcx, r8, r9, which you can memorize using the mnemonic Diane's silk dress cost $89.  
The return value register is %rax.

## Add Instruction example

Encoded representation takes on a generic form: A two byte encoding, the first byte indicates the **instruction type** and the second byte indicates the **source and destination registers**.

addq %rsi %rdi  
The encoded representation is 60 06, and the purpose of this line is to add rsi and rdi and store the result in rdi. 60 indicates the instruction type, and the 06 indicates the source register, 0, and the destination register, 6.

The effect on the state is threefold:  
1. Set the contents of %rdi to the sum of rsi and rdi   
2. Set condition flags based on the result of the sum, **OF** if result causes overflow, **ZF** if the result is zero, **SF** if the result is negative.   
3. Increment the program counter by 2, 2 because it's a 2 byte instruction

## Moving

**Y86 adds special move instructions to compensate for the lack of certain addressing modes**

### Conditional Moves

Generally referred to as cmovXX, generally in the form 2 fn rA rB.    
Encodings differ by fn     
rrmovq is a special case, even though it starts with a 2, it's still an unconditional move (register to register) code is 20, fn is 0     
What happens is dependent on the value of the condition codes.   
Conditionally copy source to destination register

## Jumps

Generally referred to as jXX, in the form 7 fn Dest   
Encodings differ by fn   
Same as x86-64 counterparts    
Encode full address of destination

## Y86 Program Stack

Region of Memory holding program data   
Top of the stack indicated by %rsp   
Stack grows toward lower addresses   
**Top element is at lowest address in stack**   
When **pushing**, must **decrement** the stack pointer    
When **popping**, **increment** stack pointer

Pushing: generally in the form pushq rA, which is in the form a 0 rA F  
This decrements the stack pointer by 8, stores the quad word from rA to memory at rsp, a is the code for pushq, and rA is the register to be pushed and F means no register   
Popping: generally in the form popq rB, which is in the form b 0 rB F   
This increments the stack pointer by 8, reads the quad word from rsp and store it in rB

**Calling** methods: generally in the form Call Dest, which is in the form 8 0 dest   
Push the address of dest onto the program stack and start executing instructions of dest

**Returning**: Popping values from stack to return is in the form 9 0  

**Advancing** the PC: nop 1 0

**Halting** the program: halt 0 0, sets the HLT flag and stops executing instructions

## Y86 Directives

**.pos** - subsequent lines of code start at the address x   
**.align** - align the next line to an x-byte boundary i.e. quad words should be divisible by 8, so align 8    
**.quad** - put x at the current address, initializing the value